In [1]:
import pandas as pd
import sqlite3
import json


In [2]:
urljson="https://api.covid19india.org/v4/min/timeseries.min.json"
df=pd.read_json(urljson)
count=0
df.head()


# for loop through the states, for loop through the dates, for loop through the delta
# first column state, second column date, then confirmed, recovered, deceased, tested, other, vaccinated
# output in a csv file and then in a SQL file


,AN,AP,AR,AS,BR,CH,CT,DL,DN,GA,...,RJ,SK,TG,TN,TR,TT,UN,UP,UT,WB
dates,"{'2020-03-26': {'delta': {'confirmed': 1}, 'de...","{'2020-03-12': {'delta': {'confirmed': 1}, 'de...","{'2020-04-02': {'delta': {'confirmed': 1}, 'de...","{'2020-03-31': {'delta': {'confirmed': 1}, 'de...","{'2020-03-22': {'delta': {'confirmed': 2}, 'de...","{'2020-03-19': {'delta': {'confirmed': 1}, 'de...","{'2020-03-19': {'delta': {'confirmed': 1}, 'de...","{'2020-03-02': {'delta': {'confirmed': 1}, 'de...","{'2020-04-09': {'delta': {'tested': 80}, 'delt...","{'2020-03-25': {'delta': {'confirmed': 3}, 'de...",...,"{'2020-03-03': {'delta': {'confirmed': 1}, 'de...","{'2020-05-04': {'delta': {'tested': 170}, 'del...","{'2020-03-02': {'delta': {'confirmed': 1}, 'de...","{'2020-03-07': {'delta': {'confirmed': 1}, 'de...","{'2020-04-06': {'delta': {'confirmed': 1}, 'de...","{'2020-01-30': {'delta': {'confirmed': 1}, 'de...","{'2020-05-19': {'delta': {'confirmed': 814}, '...","{'2020-03-04': {'delta': {'confirmed': 7}, 'de...","{'2020-03-15': {'delta': {'confirmed': 1}, 'de...","{'2020-03-17': {'delta': {'confirmed': 1}, 'de..."


In [3]:
conn=sqlite3.connect('CovidIndia.sqlite')
cur=conn.cursor()

cur.executescript('''
DROP TABLE IF EXISTS State;
DROP TABLE IF EXISTS Data;
DROP TABLE IF EXISTS State_Data;

CREATE TABLE State(
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE, name TEXT UNIQUE);
CREATE TABLE Data(
    state_id INTEGER, Date DATE, Confirmed INTEGER, Recovered INTEGER, Deceased INTEGER, Tested INTEGER, Vaccinated1 INTEGER, Vaccinated2 INTEGER);
CREATE TABLE State_Data(
    state_id INTEGER, State_Name TEXT, State_Area INTEGER, State_Population INTEGER)'''
                 )
for state in df:
    if not (state=="TT" or state=="UN"):
        for date in df[state]["dates"]:
            cur.execute('''INSERT OR IGNORE INTO State(name) VALUES(?)''',(state,))
            cur.execute('''SELECT id FROM State WHERE name=?''',(state,))
            state_id=cur.fetchone()[0]

            try:
                confirmed=df[state]["dates"][date]["delta"]["confirmed"]
            except:
                confirmed=0
                
            try:
                recovered=df[state]["dates"][date]["delta"]["recovered"]
            except:
                recovered=0
        
            try:
                death=df[state]["dates"][date]["delta"]["deceased"]
            except:
                death=0
                
            try:
                vaccinated1=df[state]["dates"][date]["delta"]["vaccinated1"]
            except:
                vaccinated1=0
                
            try:
                vaccinated2=df[state]["dates"][date]["delta"]["vaccinated2"]
            except:
                vaccinated2=0
                
            try:
                tested=df[state]["dates"][date]["delta"]["tested"]
            except:
                tested=0            
            
            cur.execute('''INSERT OR IGNORE INTO Data(state_id, Date, Confirmed, Recovered, Deceased, Tested, Vaccinated1,Vaccinated2) VALUES(?,?,?,?,?,?,?,?)''',
                        (state_id,date,confirmed,recovered,death,tested,vaccinated1,vaccinated2))

        conn.commit()




In [4]:
df1=pd.read_excel('State_Population_Data.xlsx')
cur.executescript('''
DROP TABLE IF EXISTS State_Data;

CREATE TABLE State_Data(
    state_id INTEGER, state_code TEXT, State_Name TEXT, State_Area INTEGER, State_Population INTEGER)''')

for index,row in df1.iterrows():
    cur.execute('''SELECT id FROM State WHERE name=?''',(row['State_Code'],))
    state_id=cur.fetchone()[0]
    cur.execute('''INSERT OR IGNORE INTO State_Data(state_id, state_code,State_Name, State_Area, State_Population) VALUES(?,?,?,?,?)''',
                        (state_id,row['State_Code'],row['State_Name'],row['Area'],row['Population']))
conn.commit()

In [5]:
"""js=df["AN"]["dates"]
print(js)
print(js["2020-01-30"])
print(js["2020-02-02"])
for entry in js:
    try:
        print(js[entry]['delta'])
    except:
        pass
"""

'js=df["AN"]["dates"]\nprint(js)\nprint(js["2020-01-30"])\nprint(js["2020-02-02"])\nfor entry in js:\n    try:\n        print(js[entry][\'delta\'])\n    except:\n        pass\n'